In [9]:
%load_ext autoreload

In [10]:
%autoreload

In [2]:
import sys
print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(["/home/mathia/PycharmProjects/neslearn-git/","/home/peer/PycharmProjects/neslearng/"])

Python 3.5.2 (default, Nov 17 2016, 17:05:23) 
[GCC 5.4.0 20160609] on linux


In [3]:
import matplotlib.pyplot as plt
import numpy as np
from hsa.tree_search import fceux_a_star
from tqdm import tqdm_notebook
from functools import partial
%matplotlib inline

In [4]:
from extern.fceux_learningenv.nes_python_interface.nes_python_interface import NESInterface
from hsa.machine_constants import mario_rom_location
from hsa.tree_search import heuristics

nes = NESInterface(mario_rom_location,eb_compatible=False)

In [5]:
def w_to_config(w):
    action_repeat = w[0]
    #ram, x_pos_resolution=1, x_pos_factor=1, shortcut_factor=1, height_factor=0, punishment_death=True,speed_factor=0
    heuristic_params = {"x_pos_resolution": w[1],
                        "x_pos_factor": w[2],
                        "shortcut_factor": w[2],
                        "height_factor": w[3],
                        "speed_factor": w[4]}
    heuristic = partial(heuristics.combined, **heuristic_params)
    return fceux_a_star.ConfigPack(nes, heuristic=heuristic, action_repeat=action_repeat, nr_nodes_to_expand=25)

default_w = np.array([40, 5, 1, 0, 0])

In [13]:
def w_to_r(w):
    config = w_to_config(w)
    start = fceux_a_star.make_start(nes, config.heuristic, True)
    graph = fceux_a_star.AStarGraph(config)
    best = fceux_a_star.best_first_full(start, graph, config)
    nes.restoreState(best.state)
    ram  = nes.getRAM()
    nes.render()
    progress = heuristics.combined(ram)
    return progress

In [19]:
np.random.seed(3)
nn = 4  # number of steps to take (and plot horizontally)
alpha = 0.03  # learning rate
sigma = 6  # standard deviation of the samples around current parameter vector

w = np.array(default_w)  # start point

for q in range(nn):
    # draw a population of samples in black
    noise = np.random.randn(20, w.shape[0])  # second value should be w.shape[0]
    wp = np.expand_dims(w, 0) + sigma * noise

    # draw estimated gradient as white arrow
    R = np.array([w_to_r(w) for wi in tqdm_notebook(wp)])  # do the experiments here
    R -= R.mean()
    R /= R.std()  # standardize the rewards to be N(0,1) gaussian
    g = np.dot(R, noise)
    u = alpha * g

    w += u

TypeError: Cannot cast ufunc subtract output from dtype('float64') to dtype('int64') with casting rule 'same_kind'

In [12]:
[w_to_r(w) for wi in tqdm_notebook(wp)]

[1925.0, 1925.0]

In [15]:
np.array([1925.0, 1925.0]).mean()

1925.0

In [18]:
wp

array([[ 45.36588542,   6.30952955,   1.2894924 ,  -5.59047811,
         -0.83216461],
       [ 38.93572306,   4.75177556,  -0.88100203,  -0.13145451,
         -1.43165409]])

In [5]:
np.random.randn(200, 2)

array([[ -1.35789828e+00,   5.33485331e-01],
       [  8.84767191e-01,   7.15462710e-01],
       [  1.63159694e-01,   5.48982232e-01],
       [ -1.21919001e+00,   3.19453770e-01],
       [  1.11802724e+00,   3.24890086e-01],
       [ -1.13531043e-01,   4.44032556e-01],
       [ -1.11173770e+00,   1.41814082e-01],
       [  4.17275560e-01,   1.78576431e+00],
       [ -2.29424963e+00,   4.50509031e-01],
       [  1.09410286e-02,   1.16640512e-01],
       [  1.00663668e+00,   1.33712722e+00],
       [ -1.35287760e-02,  -7.84740209e-01],
       [ -3.04215495e-01,  -4.44776856e-01],
       [  1.83519150e-02,  -4.57325806e-01],
       [  1.13967656e+00,   1.48332807e+00],
       [  1.09243523e+00,   8.98508432e-01],
       [  9.21377681e-01,   8.27451013e-01],
       [  3.89413101e-01,   7.36201393e-01],
       [  7.98114985e-01,   7.05204460e-01],
       [  1.64269758e+00,   1.29333130e+00],
       [ -6.27348251e-01,  -6.89519418e-01],
       [ -1.81920082e+00,  -1.20001001e-01],
       [ -

In [6]:
fceux_a_star.make_start(nes,w_to_config(default_w).heuristic,True)

MarioNode(id='astart', potential=8, recv_reward=0, state=array([70, 67, 83, ...,  0,  0,  0], dtype=uint8))